# How to solve power flow with gigpower
Last modified: 14 June 2021

This notebook will demonstrate how to solve powerflow with gigpower using Newton Raphson and Forward Backward Sweep. It will also show you how to compare solutions with the solution obtained by OpenDSS.

First, install gigpower.

In [ ]:
import sys
!{sys.executable} -m pip install gigpower

## Select a .dss file
Let's use some a simple feeder to demonstrate. You can find some test feeders in `./gigpower/tests/test_feeders`. This folder has modified versions of IEEE feeders.
We'll use the 13 Bus allwye as an example. 

In [ ]:
dss_file = '../gigpower/tests/test_feeders/IEEE_13_Bus_allwye.dss'

## Solve with Newton-Raphson method (NR3)
The following code block creates a SolutionNR3, calls solve, and prints solution values. 

In [ ]:

from gigpower.solution_nr3 import SolutionNR3

my_solution_nr3 = SolutionNR3(dss_file)
my_solution_nr3.solve()
my_solution_nr3.print_solution()

## Solve with Forward Backward Sweep (FBS)
Let's solve the same feeder with Forward Backward Sweep. Note that FBS only works for radial networks. The following code block creates a SolutionFBS, calls solve, and prints solution values. 

In [ ]:
from gigpower.solution_fbs import SolutionFBS

my_solution_fbs = SolutionFBS(dss_file)
my_solution_fbs.solve()
my_solution_fbs.print_solution()

## Solve with OpenDSS
`gigpower` provides a wrapper over OpenDSS.py. Let's use it to solve the same feeder using OpenDSS.

In [ ]:
from gigpower.solution_dss import SolutionDSS

my_solution_dss = SolutionDSS(dss_file)
my_solution_dss.solve()
my_solution_dss.print_solution()

## Get solution dataframes
After calling `.solve()`, can can query Solution objects for the following solution values, as Pandas DataFrames

- Bus Voltage: `Solution.get_V()`
- Bus Voltage Magnitude: `Solution.get_Vmag()`
- Line Current: `Solution.get_I()`
- Line Power, Incoming: `Solution.get_Stx()`
- Line Power, Outgoing: `Solution.get_Srx()`
- Bus Power: `Solution.get_sV()`

Using the `orient` parameter, specify `'rows'` or `'cols'` to get row-major or column-major formats.
The code block below retrieves and prints dataframes from our Solution objects, in row-major format.

In [ ]:
print('\nNR3 Voltage\n', my_solution_nr3.get_V(orient='rows'))
print('\nNR3 Voltage Magnitude\n', my_solution_nr3.get_Vmag(orient='rows'))
print('\nFBS Line Current\n', my_solution_fbs.get_I(orient='rows'))
print('\nFBS Line Power, Incoming\n', my_solution_fbs.get_Stx(orient='rows'))
print('\nOpenDSS Line Power, Outgoing\n', my_solution_dss.get_Stx(orient='rows'))
print('\nOpenDSS Bus Powers\n', my_solution_dss.get_Stx(orient='rows'))

## Compare Solutions
`gigpower.pretty_print` provides a method for printing a useful comparison between two Solution objects. The code block below compares `my_solution_nr3` with `my_solution_fbs`. 

In [ ]:
from gigpower.pretty_print import compare_solutions
compare_solutions(my_solution_nr3, my_solution_fbs, 'NR3', 'FBS')
